#Import ERA5 data using examples from:


https://www.youtube.com/watch?v=BOBUwPtxmUo

Preliminaries

In [1]:
!pip install cdsapi

  Preparing metadata (setup.py) ... done
  Created wheel for cdsapi: filename=cdsapi-0.6.1-py2.py3-none-any.whl size=12006 sha256=f9a970d219038a96acc0d3d08454dba5b148a508d93a59a55fdb44ad0a5c632f
  Stored in directory: /root/.cache/pip/wheels/7c/63/08/45461d6f6636c1aba7846828d8c787a064073945048f76d44a
Successfully built cdsapi


In [2]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00


In [ ]:
!pip install pyproj

In [ ]:
import os, sys
import pyproj
import cdsapi
from pyproj import Proj
from netCDF4 import Dataset
from datetime import datetime, timedelta

C:\Users\alden\anaconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

Creat a file in the root directory with the url and API key needed for the API

In [ ]:
#Enter your user API here
myKey = 'e91b8050-7b00-49aa-8d11-5c533352a2f3'

In [ ]:
url = 'url: https://cds.climate.copernicus.eu/api/v2'
key = 'key:' + myKey

with open('/root/.cdsapirc', 'w') as f:
    f.write('\n'.join([url, key]))

with open('/root/.cdsapirc') as f:
    print(f.read())

FileNotFoundError: [Errno 2] No such file or directory: '/root/.cdsapirc'

Assign an output filepath for the extracted data

In [ ]:
myPath = '/content/drive/My Drive/H518/Project_PDA/ERA5.nc'

#Extract data based on example code provided at:

https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form

In [ ]:
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
            'specific_humidity', 'temperature', 'u_component_of_wind',
            'v_component_of_wind',
        ],
        'pressure_level': [
            '1', '2', '3',
            '5', '7', '10',
            '20', '30', '50',
            '70', '100', '125',
            '150', '175', '200',
            '225', '250', '300',
            '350', '400', '450',
            '500', '550', '600',
            '650', '700', '750',
            '775', '800', '825',
            '850', '875', '900',
            '925', '950', '975',
            '1000',
        ],
        'year': '2022',
        'month': '12',
        'day': '31',
        'time': '23:00',
        'area': [
            45, -90, 35,
            -80,
        ],
        'format': 'netcdf',
    },
    myPath)

Read the extracted data using functionality from the NETCDF4 library

In [ ]:
data = Dataset(myPath,'r')

In [ ]:
print(data.variables.keys())

dict_keys(['longitude', 'latitude', 'level', 'time', 'q', 't', 'u', 'v'])


In [ ]:
print(data.variables['time'])

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of -2147483647 used


In [ ]:
print(data.variables['latitude'])

<class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    units: degrees_north
    long_name: latitude
unlimited dimensions: 
current shape = (41,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [ ]:
print(data.variables['q'])

<class 'netCDF4._netCDF4.Variable'>
int16 q(time, level, latitude, longitude)
    scale_factor: 1.8311248715838608e-07
    add_offset: 0.005999875239323668
    _FillValue: -32767
    missing_value: -32767
    units: kg kg**-1
    long_name: Specific humidity
    standard_name: specific_humidity
unlimited dimensions: 
current shape = (1, 37, 41, 41)
filling on


In [ ]:
print(data.variables['longitude'][:100])

[-90.   -89.75 -89.5  -89.25 -89.   -88.75 -88.5  -88.25 -88.   -87.75
 -87.5  -87.25 -87.   -86.75 -86.5  -86.25 -86.   -85.75 -85.5  -85.25
 -85.   -84.75 -84.5  -84.25 -84.   -83.75 -83.5  -83.25 -83.   -82.75
 -82.5  -82.25 -82.   -81.75 -81.5  -81.25 -81.   -80.75 -80.5  -80.25
 -80.  ]


In [ ]:
print(data.variables['latitude'][:100])

[45.   44.75 44.5  44.25 44.   43.75 43.5  43.25 43.   42.75 42.5  42.25
 42.   41.75 41.5  41.25 41.   40.75 40.5  40.25 40.   39.75 39.5  39.25
 39.   38.75 38.5  38.25 38.   37.75 37.5  37.25 37.   36.75 36.5  36.25
 36.   35.75 35.5  35.25 35.  ]


#Formatting the date time values

In [ ]:
print(data.variables['time'][:])

[1078199]


In [ ]:
unit = data.variables['time'].units
print(unit)

hours since 1900-01-01 00:00:00.0


In [ ]:
t = int(data.variables['time'][:])
refDate = datetime(int(unit[12:16]),int(unit[17:19]),int(unit[20:22]))
print(refDate)

1900-01-01 00:00:00


In [ ]:
starting_date = refDate + timedelta(hours=t)
print(starting_date)

2022-12-31 23:00:00


In [ ]:
print(data.variables['v'][:1])

[[[[ 3.15286569e+01  3.13410345e+01  3.11887970e+01 ...  2.88607976e+01
     2.85637287e+01  2.82518476e+01]
   [ 3.12241819e+01  3.10678299e+01  3.09386337e+01 ...  2.84509907e+01
     2.81267660e+01  2.77901977e+01]
   [ 3.09386337e+01  3.08020314e+01  3.06580230e+01 ...  2.79934553e+01
     2.76577099e+01  2.73450058e+01]
   ...
   [ 1.88526221e+01  1.89151629e+01  1.88806009e+01 ...  1.57667268e+01
     1.56852592e+01  1.55173865e+01]
   [ 1.91924820e+01  1.91538055e+01  1.90246093e+01 ...  1.53799613e+01
     1.51265064e+01  1.48063962e+01]
   [ 1.93019284e+01  1.91612116e+01  1.89546624e+01 ...  1.45759827e+01
     1.42048524e+01  1.38024516e+01]]

  [[ 3.24420819e+01  3.22659802e+01  3.20849410e+01 ...  3.40615597e+01
     3.42738693e+01  3.44927622e+01]
   [ 3.17409665e+01  3.15887290e+01  3.14768139e+01 ...  3.45125119e+01
     3.47289360e+01  3.49264333e+01]
   [ 3.11237875e+01  3.10884025e+01  3.11295478e+01 ...  3.48894026e+01
     3.50375255e+01  3.51510865e+01]
   ...
   